In [ ]:
from bs4 import BeautifulSoup
import requests
import lxml
import os
import csv
import pandas as pd
import plotly.express as px
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import *
import string
from nltk.corpus import stopwords
import re
from collections import Counter
from functools import reduce
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import math
import heapq as hq

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
url2='https://www.atlasobscura.com/places?page='
sort='&sort=likes_count'

### 1.1. Get the list of places And 1.2. Crawl places

We create the url txt file and html folders

In [ ]:
def crawl():
    pathAncestor = os.path.join("./", "htmlpages") # Path
    #os.mkdir(pathAncestor) # create the folder in the path
    
    f = open("urlpages.txt", "w") #create txt file with urls
    count=1 #counting html files
    headpart='https://www.atlasobscura.com'
    
    for i in range(1, 401):
        os.makedirs(os.path.join(pathAncestor, 'page ' + str(i))) #create folder of html files
        
        url = url2 +str(i) +sort
        page_1 = requests.get(url)
        soup = BeautifulSoup(page_1.content, features='lxml')
        
        for a in soup.find_all('a', class_="content-card content-card-place"): #the class where the url is
            save=a.get('href')
            f.write(save +'\n')
            
            subdirectory = pathAncestor + "/page " + str(i) # select the corresponding folder to insert the html article
            article_name = "/article_"+str(count)+".html" 
            count+=1
            complete_path = subdirectory + article_name # insert the new complete path where create the html file
            
            with open(complete_path, "wb") as htmlFile:
                link = headpart + save
                try:
                    page_2 = requests.get(link) # request the page
                except:
                    with open("failureRequest.txt", "a") as error: # if we loose the request place, we put into a file the link doesn't download well, then we set the "urlpages.txt" with these link
                        error.write(link)
                        error.close()

                soup = BeautifulSoup(page_2.text, features='lxml')

                htmlFile.write(soup.encode('utf-8'))
                htmlFile.close()

    f.close()

In [ ]:
crawl()

### 1.3 Parse downloaded pages


We create the tsv files for each html file

In [ ]:
def scrap(tsv_writer, article): 
    
    with open(article, 'r', encoding="utf-8") as out_file: # for each html article downloaded scrape it!
        contents = out_file.read()
        soup = BeautifulSoup(contents, features="lxml") #parse the text
        
    try:
        roi_header = soup.find_all("div", {"class": "DDPage__header-container grid-row"})[0]
        placeName = roi_header.find("h1", {"class":"DDPage__header-title"}).text.strip()

    except:
        placeName = " "
    try:
        related_list = soup.find("div",{
            "class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links",
            "data-gtm-template":"DDP Footer Recirc Lists"
        })
        
        placeRelatedLists = [data.find("span").text.strip() for data in related_list.find_all("h3")]
    except:
        placeRelatedLists=''
        
    try:
        related_places = soup.find("div",{
            "class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links",
            "data-gtm-template":"DDP Footer Recirc Related"
        })
        placeRelatedPlaces = [data.find("span").text.strip() for  data in related_places.find_all("h3")]
    except:
        placeRelatedPlaces=''
    try:
        placeDesc = soup.find_all("div", {"class": "DDP__body-copy"})[0]
        placeDesc = "".join([p.text.strip() for p in placeDesc.find_all("p")])
    except:
        placeDesc=''
    try:
        location = roi_header.find("div", {"class":"DDPage__header-place-location"}).text.strip()
    except:
        location=''
    try:
        counters = roi_header.find_all("div",{"class":"title-md item-action-count"})
        numPeopleVisited=int(counters[0].text.strip())
    except:
        numPeopleVisited=''
    try:
        numPeopleWant = int(counters[1].text.strip())
    except:
        numPeopleWant=''
    try:
        placeShortDesc = roi_header.find("h3", {"class":"DDPage__header-dek"}).text.strip()
    except:
        placeShortDesc=''
    try:
        sidebar = soup.find("div", {"class":"DDPageSiderail"})
        nearby_places = sidebar.find_all("div",{"class":"DDPageSiderailRecirc__item-title"})
        positions =  sidebar.find("div",{"class":"DDPageSiderail__coordinates js-copy-coordinates"}).attrs["data-coordinates"].split(",")
        placeAlt= float(positions[0].strip())
    except:
        placeAlt=''
    try:
        placeLong = float(positions[1].strip())
    except:
        placeLong=''
    try:
        placeNearby = [place.text for place in nearby_places]
    except:
        placeNearby=''
    try:
        placeAddress = [x.text for x in soup.find_all('div', {'class': 'DDPage__header-place-location'})][0]
        
    except:
        placeAddress=''
        
    try:
        footer = soup.find("div", {"id":"ugc-module"})
        Editors = footer.find_all("a",{"class":"DDPContributorsList__contributor"})
        placePubDate = footer.find("div",{"class":"DDPContributor__name"}).text
        placePubDate = datetime.strptime(placePubDate,"%B %d, %Y")
    except:
        placePubDate=''
    try:
        #10-placeEditors, 11-placePubDate, 2-placeTags
        placeEditors = [editors.find("span").text if editors.find("span") else editors.text for editors in Editors]
    except:
        placeEditors=''
    try:
        placeTags = [item.get_text("").replace("\n","") for item in soup.find_all("a",{"class":"itemTags__link js-item-tags-link"})]
        # set_trace(header="sidebar")
    except:   
        placeTags=''
    placeURL=''

    tsv_writer.writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, 
                                placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt,
                                placeLong, placeEditors, placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL])


In [ ]:
def create_tsv(input_path):
    path = str("./"+input_path)

    filenames = os.listdir(path)
    for i in range(1, 401):
        filenames = os.listdir(path + '/page ' + str(i))

        for file in filenames:
            with open(path + '/page ' + str(i) + '/article_'+str(file.split("_")[1].replace(".html", ""))+'.tsv', 'w', encoding="utf-8", newline='') as out_file: # create for each html article its article_i.tsv 
                tsv_writer = csv.writer(out_file, delimiter='\t')
                tsv_writer.writerow(['placeName', 'placeTags', 'numPeopleVisited', 'numPeopleWant', 
                                'placeDesc', 'placeShortDesc', 'placeNearby', 'placeAddress', 'placeAlt',
                                'placeLong', 'placeEditors', 'placePubDate','placeRelatedLists','placeRelatedPlaces','placeURL'])
                scrap(tsv_writer, path + '/page ' + str(i) + "/" + file)

In [ ]:
create_tsv('htmlpages')

We create the tsv file 

In [ ]:
def dataFr():
    path = "./htmlpages"
    data = pd.DataFrame()

    for num in range(1, 401):
        filenames = os.listdir(path + '/page ' + str(num))
        for file in filenames:
            if file.endswith(".tsv"): # look for tsv files
                with open(path + '/page ' + str(num) + '/article_'+str(file.split("_")[1]), 'r', encoding="utf-8", newline='') as file_tsv:
                        df = pd.read_csv(file_tsv,sep = "\t")
                        data = pd.concat([data,df])
                            
    data.to_csv('data.tsv', sep='\t', index=False)


In [ ]:
dataFr() #we call the tsv function to crete our dataset

I forgot the url field

In [ ]:
data=pd.read_csv('data.tsv', sep='\t')

In [ ]:
with open ('urlpages.txt', 'r') as url_file:
    for url in url_file:
        url2=url.replace('\n','').replace('/',' ').split(' ')[2].replace('-',' ')
        data.loc[data['placeName'].str.lower() == url2, 'placeURL'] = url.replace('\n','')
        

In [ ]:
data.to_csv('data2.tsv', sep='\t', index=False)

In [ ]:
data.head()

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,Leadenhall Market,"['markets', 'film locations', 'film']",3158.0,7644.0,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England",51.5126,-0.0834,"['Meg', 'philoursmars', 'Gavin', 'jules 1f6b22...",NaN,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",/places/leadenhall-market
1,Underground Tunnels of Los Angeles,"['tunnels', 'subterranean sites', 'architectur...",660.0,6445.0,While the rest of the country was forced to go...,"During prohibition, corrupt city officials ran...","['Cathedral of Our Lady of the Angels', 'Trifo...","Los Angeles, California",34.0558,-118.2447,"['Rachel', 'ryannrues', 'MayorMyrrh', 'Avoidin...",NaN,"[""The World's Top 100 Wonders in 2018"", ""Los A...","['Tower of Wind', 'Catacombs of Washington, D....",NaN
2,Dream House,"['optical oddities', 'eccentric homes', 'music...",769.0,6019.0,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","['Aretha Franklin Subway Tributes', 'Farm.One'...","Manhattan, New York",40.7185,-74.0048,"['seanmattison', 'Sofy', 'Gray', 'gidwyz', 'mb...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Callejon de Hamel', 'Casa Neverlandia', 'Cas...",/places/dream-house
3,The Wave Organ,"['sea organ', 'aletrail', 'sounds', 'music', '...",2435.0,7441.0,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","San Francisco, California",37.8085,-122.4401,"['mbison', 'routt1111', 'bylow1md', 'wefiddleb...",NaN,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",NaN
4,The Ruins of St. Dunstan-in-the-East,"['gardens', 'world war ii', 'churches', 'plant...",1891.0,5786.0,The church of St.Dunstan-in-the-East has survi...,One of the few remaining casualties of the Lon...,"['Billingsgate Roman House and Baths', 'Philpo...","London, England",51.5097,-0.0827,"['Luke J Spencer', 'vrobbins', 'JZA', 'LorriAd...",NaN,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Kaiser Wilhelm Memorial Church', 'Ruins of H...",NaN


In [ ]:
data.info()
data=data.dropna(subset=['placeDesc'])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   placeName           378 non-null    object 
 1   placeTags           378 non-null    object 
 2   numPeopleVisited    365 non-null    float64
 3   numPeopleWant       365 non-null    float64
 4   placeDesc           365 non-null    object 
 5   placeShortDesc      365 non-null    object 
 6   placeNearby         365 non-null    object 
 7   placeAddress        365 non-null    object 
 8   placeAlt            365 non-null    float64
 9   placeLong           365 non-null    float64
 10  placeEditors        365 non-null    object 
 11  placePubDate        0 non-null      float64
 12  placeRelatedLists   260 non-null    object 
 13  placeRelatedPlaces  365 non-null    object 
 14  placeURL            206 non-null    object 
dtypes: float64(5), object(10)
memory usage: 44.4+ KB


### 2.1. Conjunctive query

clean the description field

In [ ]:
def cleanWords(field):
    stemmer = PorterStemmer()

    snowstem = SnowballStemmer('english')
    new_w=[snowstem.stem(word) for word in field.split(' ')] #stem the words
    new_w = list(map(lambda x: x.replace(x, x.translate(str.maketrans('', '', string.punctuation))), new_w)) #remove punctuation  
    stop_words = set(stopwords.words('english'))
    new_w= list(filter(lambda word: word not in stop_words, new_w)) #remove stop words
    new_w=list(map(lambda x: re.sub('[\W_]+', '', x),new_w)) #remove everything is not letters or number 
    
    return new_w

In [ ]:
data['placeDesc_new'] = data.placeDesc.apply(lambda x: cleanWords(x))

In [ ]:
data.head(10)

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL,placeDesc_new
0,Leadenhall Market,"['markets', 'film locations', 'film']",3158.0,7644.0,The ornate 19th-century painted roof and cobbl...,This ornate Victorian marketplace was the sett...,"['The Cornhill Devils ', ""London's Original an...","London, England",51.5126,-0.0834,"['Meg', 'philoursmars', 'Gavin', 'jules 1f6b22...",NaN,"['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",/places/leadenhall-market,"[ornat, 19thcenturi, paint, roof, cobbl, floor..."
1,Underground Tunnels of Los Angeles,"['tunnels', 'subterranean sites', 'architectur...",660.0,6445.0,While the rest of the country was forced to go...,"During prohibition, corrupt city officials ran...","['Cathedral of Our Lady of the Angels', 'Trifo...","Los Angeles, California",34.0558,-118.2447,"['Rachel', 'ryannrues', 'MayorMyrrh', 'Avoidin...",NaN,"[""The World's Top 100 Wonders in 2018"", ""Los A...","['Tower of Wind', 'Catacombs of Washington, D....",NaN,"[rest, countri, forc, go, dry, underneath, dow..."
2,Dream House,"['optical oddities', 'eccentric homes', 'music...",769.0,6019.0,"When walking down Church Street in Tribeca, ke...","La Monte Young and Marian Zazeela's ""Dream Hou...","['Aretha Franklin Subway Tributes', 'Farm.One'...","Manhattan, New York",40.7185,-74.0048,"['seanmattison', 'Sofy', 'Gray', 'gidwyz', 'mb...",NaN,['30 Unexpected Places to Have a Joyful Advent...,"['Callejon de Hamel', 'Casa Neverlandia', 'Cas...",/places/dream-house,"[walk, church, street, tribeca, keep, eye, bla..."
3,The Wave Organ,"['sea organ', 'aletrail', 'sounds', 'music', '...",2435.0,7441.0,"Located on a jetty in the San Francisco Bay, t...",A huge musical instrument played by the ocean.,"['Long Now Orrery', 'The Stern of the Briganti...","San Francisco, California",37.8085,-122.4401,"['mbison', 'routt1111', 'bylow1md', 'wefiddleb...",NaN,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",NaN,"[locat, jetti, san, francisco, bay, wave, orga..."
4,The Ruins of St. Dunstan-in-the-East,"['gardens', 'world war ii', 'churches', 'plant...",1891.0,5786.0,The church of St.Dunstan-in-the-East has survi...,One of the few remaining casualties of the Lon...,"['Billingsgate Roman House and Baths', 'Philpo...","London, England",51.5097,-0.0827,"['Luke J Spencer', 'vrobbins', 'JZA', 'LorriAd...",NaN,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Kaiser Wilhelm Memorial Church', 'Ruins of H...",NaN,"[church, stdunstanintheeast, surviv, lot, dure..."
5,Secret Tiled Staircase,"['stairs', 'outsider art', 'architectural oddi...",2012.0,6331.0,The 16th Avenue Staircase is 163 steps of mosa...,An artsy hidden staircase leads to breathtakin...,['Helen Crocker Russell Library of Horticultur...,"San Francisco, California",37.7563,-122.4735,"['rebeccaclara', 'tylercole', 'Rosa', 'tress22...",NaN,"[""Leonardo Nam's 16 Quirky Roadside Attraction...","[""Glasgow City Chambers' Staircase"", 'Fortress...",/places/secret-tiled-staircase,"[16th, avenu, staircas, 163, step, mosaic, til..."
6,Highgate Cemetery,"['rivals week', 'vampires', 'hoaxes', 'occult'...",2637.0,8233.0,"Opened in 1839, Highgate is one of London’s mo...",London's creepiest cemetery was once the site ...,"[""World's Largest Potted Plant"", 'Dick Whittin...","London, England",51.5675,-0.1483,"['Lauren Levesque', 'KrystalPeach', 'littlebru...",NaN,"[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",/places/highgate-cemetery,"[open, 1839, highgat, one, london, infam, ceme..."
7,The Echo Park Time Travel Mart,['purveyors of curiosities'],1506.0,5784.0,Have you ever traveled through time and foolis...,Non-profit time travel mart sells goods you wo...,"['Angelus Temple', 'Tom House', 'Ca

We create the vocabulary

In [ ]:
vocabolary=Counter(reduce(lambda x, y: x + y, data.placeDesc_new))

In [ ]:
word_dict = {} #dictionary with word as key and id as value
item_id = 1
for word in vocabolary.keys():
    word_dict[word] = item_id
    item_id += 1

we create th inverted index

In [ ]:
inverted_idx = {}
for word, item_id in word_dict.items():
    inverted_idx[item_id] = list(data[data.placeDesc_new.apply(lambda row: word in row)].index)

In [ ]:
snowstem = SnowballStemmer('english')
def stem_text(txt):
    return [snowstem.stem(word) for word in txt.split(' ')]

In [ ]:
def serch(query_str):
    data_conj=pd.DataFrame(columns=['placeName', 'placeDesc', 'placeURL'])
    results = [set(inverted_idx[item]) for item in [word_dict[word] for word in stem_text(query_str)]]
    conjuctive=set(reduce(lambda x,y: x.intersection(y), results))
    for doc in conjuctive:
        new_row = {'placeName': data.loc[doc]['placeName'], 'placeDesc': data.loc[doc]['placeDesc'], 'placeURL': data.loc[doc]['placeURL']}
        #append row to the dataframe
        
        data_conj = data_conj.append(new_row, ignore_index=True)
    data_conj=data_conj.dropna(subset=['placeName'])
    return data_conj
        


In [ ]:
serch('america museum')

,placeName,placeDesc,placeURL
0,Mütter Museum,Located inside the headquarters of the College...,NaN
1,National Bonsai Museum,The National Bonsai Museum isn’t the most heav...,/places/national-bonsai-museum
2,Cryptozoology & Paranormal Museum,"The spookiest, most unexplainable phenomena al...",NaN
3,Shofuso Japanese House and Garden,Near the banks of the Schuylkill River lie a g...,/places/shofuso-japanese-house-and-garden
4,CDC Museum,"In the 1995 hit film Outbreak, residents of th...",/places/cdc-museum
5,The Churchill War Rooms,Hidden deep underneath the streets of Westmins...,NaN
6,Museum of Sex,It’s amazing that it took so long for New York...,NaN
7,Salvador Dalí Museum,The world’s largest collection of art by Salva...,NaN
8,The Natural Bridge,Often cited as having a place among the great ...,NaN


### 2.2) Conjunctive query & Ranking score

2.2.1) Inverted index

In [ ]:
#tf t,d = numero di parole /lunghezza campo
#idf t = log(N/df)  N is the total number of documents in the collection, and df is the number of documents a term occurs in. 
#w t,d = tf * idf The TF-IDF is the dot product of the TF and IDF, so calculated as:
inverted_idx2 = {}
term_idf = {}

for word, item_id in word_dict.items():
    result = []
    docList = inverted_idx[item_id]
    idf = math.log(len(data)/len(docList))
    for doc in docList:
        tf = (data.loc[doc].placeDesc.count(word))/len(data.loc[doc].placeDesc)
        tfidf = tf * idf
        result.append((doc, tfidf))
    term_idf[item_id] = idf
    inverted_idx2[item_id] = result

In [ ]:
def cos_sim(query, doc):

    query = query.strip().split()
    
    query_tfidf = {}
    tf = 1/len(query)
    for word in query:
        query_tfidf[word_dict[word]] = term_idf[word_dict[word]] * tf
    
    dot_product = 0
    query_norm = 0
    for item_id, q_tfidf in query_tfidf.items():
        query_norm += q_tfidf ** 2
        doc_tfidf = [pair[1] for pair in inverted_idx2[item_id] if pair[0] == doc] # either [tfidf_{term,doc}] or []
        if doc_tfidf:
            dot_product += q_tfidf * doc_tfidf[0]

    doc_norm = 0
    for item_id, docList in inverted_idx2.items():
        tfidf = [pair[1] for pair in docList if pair[0] == doc]
        if tfidf:
            doc_norm += tfidf[0] ** 2
    doc_norm = doc_norm ** 0.5

    return dot_product / (query_norm * doc_norm)

2.2.2) Execute the query

In [ ]:
def serch2(query_str):
    results = [set(inverted_idx[item]) for item in [word_dict[word] for word in stem_text(query_str)]]
    conjuctive=set(reduce(lambda x,y: x.intersection(y), results))
    data_conj = data.loc[conjuctive]\
                .sort_index(key=lambda docs : [cos_sim(query_str, doc) for doc in docs],
                            ascending=False)\
                [['placeName', 'placeDesc', 'placeURL']]

    return data_conj

In [ ]:
serch2('america museum')

,placeName,placeDesc,placeURL
108,Shofuso Japanese House and Garden,Near the banks of the Schuylkill River lie a g...,/places/shofuso-japanese-house-and-garden
145,The Churchill War Rooms,Hidden deep underneath the streets of Westmins...,NaN
365,CDC Museum,"In the 1995 hit film Outbreak, residents of th...",/places/cdc-museum
264,Cryptozoology & Paranormal Museum,"The spookiest, most unexplainable phenomena al...",NaN
319,The Natural Bridge,Often cited as having a place among the great ...,NaN
32,Mütter Museum,Located inside the headquarters of the College...,NaN
285,Salvador Dalí Museum,The world’s largest collection of art by Salva...,NaN
290,National Bonsai Museum,The National Bonsai Museum isn’t the most heav...,/places/national-bonsai-museum
119,Museum of Sex,It’s amazing that it took so long for New York...,NaN


### 4. Visualizing the most relevant places

We use mapbox to visualize the places on a map, I know this works I tried in another dataset and not the result of the third exercise

In [ ]:
def maps(data):  

    fig = px.scatter_mapbox(data_prova, lat="placeAlt", lon="placeLong", hover_name="placeName", hover_data=["numPeopleWant", "numPeopleVisited", "placeAddress"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()